## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.88,147.33,43.00,80,20,8.05,few clouds
1,1,Dalbandin,PK,28.89,64.41,57.92,28,0,2.57,clear sky
2,2,Qaanaaq,GL,77.48,-69.36,-12.95,71,0,7.56,clear sky
3,3,Kapaa,US,22.08,-159.32,80.60,83,40,10.29,moderate rain
4,4,Bethel,US,41.37,-73.41,73.40,35,1,4.70,clear sky


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.08,-159.32,80.60,83,40,10.29,moderate rain
7,7,Bengkulu,ID,-3.80,102.27,76.14,86,100,4.12,overcast clouds
9,9,Madimba,TZ,-10.40,40.33,78.80,83,20,7.61,few clouds
13,13,Yarada,IN,17.65,83.27,80.60,78,40,2.24,mist
22,22,Port Blair,IN,11.67,92.75,81.03,79,0,7.00,light rain
23,23,Georgetown,MY,5.41,100.34,78.80,88,40,3.36,scattered clouds
25,25,Luganville,VU,-15.53,167.17,80.60,88,40,1.12,scattered clouds
28,28,Lucea,JM,18.45,-78.17,77.00,94,90,5.82,light rain
45,45,Cabo San Lucas,MX,22.89,-109.91,84.00,69,75,10.29,broken clouds
51,51,Nanakuli,US,21.39,-158.15,80.60,78,40,8.05,scattered clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                204
City                   204
Country                203
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,80.60,moderate rain,22.08,-159.32,
7,Bengkulu,ID,76.14,overcast clouds,-3.80,102.27,
9,Madimba,TZ,78.80,few clouds,-10.40,40.33,
13,Yarada,IN,80.60,mist,17.65,83.27,
22,Port Blair,IN,81.03,light rain,11.67,92.75,
23,Georgetown,MY,78.80,scattered clouds,5.41,100.34,
25,Luganville,VU,80.60,scattered clouds,-15.53,167.17,
28,Lucea,JM,77.00,light rain,18.45,-78.17,
45,Cabo San Lucas,MX,84.00,broken clouds,22.89,-109.91,
51,Nanakuli,US,80.60,scattered clouds,21.39,-158.15,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
       
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [55]:
# 7. Drop the rows where there is no Hotel Name.
hotel_mask = hotel_df["Hotel Name"] == ""
hotel_df.loc[~hotel_mask]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,80.60,moderate rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
7,Bengkulu,ID,76.14,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
13,Yarada,IN,80.60,mist,17.65,83.27,"D-26/2, Dolphin Hill,"
22,Port Blair,IN,81.03,light rain,11.67,92.75,Fortune Resort Bay Island
23,Georgetown,MY,78.80,scattered clouds,5.41,100.34,Cititel Penang
...,...,...,...,...,...,...,...
662,Biltine,TD,78.58,clear sky,14.53,20.92,Hôtel Trois Étoiles de Biltine
665,Ixtapa,MX,86.00,clear sky,20.70,-105.20,Container Inn Puerto Vallarta
668,Kuching,MY,77.00,light rain,1.55,110.33,Hilton Kuching
674,Cozumel,MX,84.99,broken clouds,20.51,-86.95,Cozumel Palace® All Inclusive Resort


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [64]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd{Current Description}</dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))